In [ ]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import time
import random
from imutils import paths
from sklearn.utils import shuffle

In [ ]:
print("TF version:", tf.__version__)
print("Keras version:", tf.keras.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

In [ ]:
#ROOT_DIR = "/content/drive/MyDrive/CASGEM-Egitim/Egitim-Part2/Day11-DeepLearning/notebooks/"
ROOT_DIR = "https://media.githubusercontent.com/media/yapay-ogrenme/casgem-eu-project-training-on-data-mining-2nd/main/PART2/Day11-DeepLearning/notebooks/"

MODEL_PATH = ROOT_DIR + "models/"
MODEL_FILEPATH = MODEL_PATH + 'mobilenet_fine_trashnet_120.h5'

DATASET_FOLDER_PATH = "https://github.com/yapay-ogrenme/casgem-eu-project-training-on-data-mining-2nd/blob/main/PART2/Day11-DeepLearning/notebooks/datasets/waste_sorting/waste_sorting_images.zip?raw=true" #waste_sorting_images.zip

In [ ]:
!wget $MODEL_FILEPATH
!wget $DATASET_FOLDER_PATH

In [ ]:
!unzip "waste_sorting_images.zip?raw=true" -d "waste_sorting"

## Model Load

In [ ]:
#Model
num_classes=6

base_model = tf.keras.applications.MobileNet(weights='imagenet', include_top=False, classes=num_classes)

x = tf.keras.layers.GlobalAveragePooling2D(input_shape=base_model.output_shape[1:])(base_model.output)
x = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_initializer='glorot_normal')(x)
model = tf.keras.Model(inputs=base_model.input, outputs=x)

#load weights
model.load_weights("mobilenet_fine_trashnet_120.h5")

In [ ]:
model.summary()

# MY Gradient class activation map (GRAD CAM)

## Prediction

In [ ]:
#my_test_path = "waste_sorting/images/glass.jpeg"
my_test_path = "waste_sorting/plastic471.jpg"
#my_test_path = "waste_sorting/20190816_103719.jpg"
#my_test_path = "waste_sorting/cardboard323.jpg"

target_size = (224, 224)

img = cv2.imread(my_test_path)
img = cv2.resize(img, target_size)

plt.imshow(img)
plt.axis('off')
plt.show()

img = np.array(img, dtype="float") / 255.0
img_data = np.expand_dims(img, axis=0)

result_pred = model.predict(img_data, batch_size=1) 
print(result_pred)

labels = ["cardboard", "glass", "metal", "paper", "plastic", "trash"]
bestclass = labels[np.argmax(result_pred, axis=1)[0]]
print("bestclass", bestclass)

argmax = np.argmax(result_pred, axis=1)[0]
print(argmax)

## GRAD CAM

In [ ]:
tf.compat.v1.disable_eager_execution()

output = model.output[:, argmax]

last_conv_layer = model.get_layer('conv_pw_13')
grads = tf.keras.backend.gradients(output, last_conv_layer.output)[0]
pooled_grads = tf.keras.backend.mean(grads, axis=(0, 1, 2))
iterate = tf.keras.backend.function([model.input], [pooled_grads, last_conv_layer.output[0]])

pooled_grads_value, conv_layer_output_value = iterate([img_data])

    
for i in range(last_conv_layer.output.shape[-1]):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]
    
    
heatmap = np.mean(conv_layer_output_value, axis=-1)
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)


heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
hif = .8
superimposed_img = heatmap * hif + img

output = 'waste_sorting/gradcam_result.jpg'

cv2.imwrite(output, superimposed_img)

img_output = mpimg.imread(output)

plt.imshow(img_output)
plt.axis('off')
plt.title(bestclass.upper())